In [196]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from nltk import RegexpParser, Tree
import pandas as pd
import numpy as np
import ast

In [197]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos", max_heap_size='-Xmx500m')

def word_segment(text):
    annotated_text = annotator.annotate(text)['sentences'][0]
    return [dic['form'].replace('_',' ').lower() for dic in annotated_text]

In [198]:
data = pd.read_csv('./data/data_labeled_updated.csv').drop('Unnamed: 0', axis=1)
ner_feature = pd.read_csv('./data/ner_title_feature.csv')

In [199]:
data.head()

,NewsTitle,NewsFullContent,label
0,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...,"[('ông', 0), ('nguyễn hoà bình chủ tịch', 0), ..."
1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...,"[('trong', 0), ('khi', 0), ('thị trường', 1), ..."
2,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...,"[('kế hoạch', 1), ('này', 0), ('đã', 0), ('đượ..."
3,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...,"[('theo', 1), ('đó', 0), ('tháng', 1), ('7/200..."
4,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,"[('trong', 0), ('khi', 0), ('chỉ', 1), ('có', ..."


In [200]:
ner_feature.head()

,Unnamed: 0,name_entity,title_tag
0,0,"[False, False, False, False, True, False, Fals...","[False, False, False, False, True, False, Fals..."
1,1,"[False, False, False, True, False, False, Fals...","[False, False, False, False, False, True, Fals..."
2,2,"[False, False, False, False, False, True, Fals...","[False, False, False, False, False, True, Fals..."
3,3,"[False, False, False, False, True, True, False...","[False, False, False, False, True, True, False..."
4,4,"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."


In [201]:
data['list_label'] = data['label'].apply(lambda x: ast.literal_eval(x))
def get_y(text):
    return [word[1] for word in text]



In [202]:
X=data.drop('label', axis=1)
X = pd.concat([X,ner_feature], axis=1)
y = data['list_label'].apply(lambda x: get_y(x)).to_list()

In [203]:
X = X.drop('Unnamed: 0', axis=1)

In [204]:
def convertBoolArr(arr):
    return [0 if x.lstrip() == 'False' else 1 for x in arr[1:-1].split(',')]

X['name_entity'] = X['name_entity'].apply(convertBoolArr)
X['title_tag'] = X['title_tag'].apply(convertBoolArr)

In [205]:
X.head()

,NewsTitle,NewsFullContent,list_label,name_entity,title_tag
0,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...,"[(ông, 0), (nguyễn hoà bình chủ tịch, 0), (hđq...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...,"[(trong, 0), (khi, 0), (thị trường, 1), (châu ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...,"[(kế hoạch, 1), (này, 0), (đã, 0), (được, 1), ...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...,"[(theo, 1), (đó, 0), (tháng, 1), (7/2009, 0), ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,"[(trong, 0), (khi, 0), (chỉ, 1), (có, 1), (5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [206]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NewsTitle        649 non-null    object
 1   NewsFullContent  649 non-null    object
 2   list_label       649 non-null    object
 3   name_entity      649 non-null    object
 4   title_tag        649 non-null    object
dtypes: object(5)
memory usage: 25.5+ KB


In [207]:
X_train_process = X['NewsFullContent'].apply(lambda x: annotator.annotate(str(x)))

In [208]:
def get_postag(x):
    pos = []
    for dic in x['sentences'][0]:
        pos.append(dic['posTag'])
    return pos

In [209]:
def get_form(x):
    pos = []
    for dic in x['sentences'][0]:
        pos.append(dic['form'])
    print(pos)
    return pos

In [210]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [211]:
X['form'] = X_train_process.apply(lambda x: get_form(x))
X['pos_tag'] = X_train_process.apply(lambda x: get_postag(x))

'riêng', 'của', 'từng', 'phân', 'khúc', 'khách_hàng', 'Biên', 'lợi_nhuận', 'gộp', '10', 'tháng', 'của', 'Công_ty', 'nằm', 'ở', 'mức', '19', '7%', 'Bên', 'cạnh', 'đó', 'thương_hiệu', 'Style', 'by', 'PNJ', 'vừa', 'ra_mắt', 'hứa_hẹn', 'sẽ', 'đẩy', 'nhanh', 'đà', 'tăng_tốc', 'và', 'góp_phần', 'quan_trọng', 'giúp', 'PNJ', 'về', 'đích', 'sớm', 'trong', 'tháng', '11/2020', 'Ngoài_ra', 'trong', '10', 'tháng', 'đầu', 'năm', '2020', 'PNJ', 'đã', 'mở', 'mới', '26', 'cửa_hàng', 'hoàn_thành', '84%', 'kế_hoạch', 'và', 'nâng_cấp', '8', 'cửa_hàng', 'bên', 'cạnh', 'đó', 'là', 'chuyển_đổi', 'một', 'cửa_hàng', 'silver', 'theo', 'định_hướng', 'tối_ưu_hoá', 'doanh_thu', '/', 'cửa_hàng', 'Với', 'tốc_độ', 'phát_triển', 'và', 'vị_thế', 'đạt', 'được', 'theo', 'đánh_giá', 'của', 'Forbes', 'Việt_Nam', 'giá_trị', 'thương_hiệu', 'của', 'PNJ', 'trong', 'năm', '2020', 'đạt', '94', '1', 'triệu', 'USD', 'tăng', '15', '5', 'triệu', 'USD', 'so', 'với', 'kỳ', 'đánh_giá', '2019', 'thăng', '3', 'hạng', 'lên', 'vị_trí', '21

In [212]:
X['form_text'] = X['form'].apply(lambda x: (" ").join(x))

In [213]:
X['form_text']

0      Ông Nguyễn_Hoà_Bình_Chủ_tịch HĐQT của VCB cho ...
1      Trong khi thị_trường Châu_Âu có phiên tăng điể...
2      Kế_hoạch này đã được đại_diện PNJ bà Cao_Thị_N...
3      Theo đó tháng 7/2009 Đại_học FPT sẽ chính_thức...
4      Trong khi chỉ có 5 công_ty nắm tới trên 30 ngh...
                             ...                        
644    Ông Nguyễn_Phi_Long_PCT UBND tỉnh Bình_Định vừ...
645    Ông Nguyễn_Phi_Long_PCT UBND tỉnh Bình_Định vừ...
646    Giá phát_hành dự_kiến là 20 000 đồng / cp thấp...
647    Đại_diện UBND Bắc_Giang đóng_góp ý_kiến thiết_...
648    Công_ty hoàn_thành 88 7% kế_hoạch lợi_nhuận tr...
Name: form_text, Length: 649, dtype: object

In [214]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [215]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train['form_text'])
feature_names = tfidf.get_feature_names()

def get_ifidf_for_words(text):
    tfidf_matrix= tfidf.transform([text]).todense()
    feature_index = tfidf_matrix[0,:].nonzero()[1]
    tfidf_scores = zip([feature_names[i] for i in feature_index], [tfidf_matrix[0, x] for x in feature_index])
    return dict(tfidf_scores)

In [216]:
X_train['ifidf_for_words'] = X_train['form_text'].apply(lambda x : get_ifidf_for_words(x))

In [217]:
def get_tfidf_for_word(word, dic):
    if word in dic.keys():
        return dic[word]
    else:
        return 0

def get_tfidf_for_docs(text, dic):
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']
    tfidf_words = text_df['word'].apply(lambda x: get_tfidf_for_word(x, dic))
    return tfidf_words

In [218]:
tfidf_docs = []
for i,row in X_train[['form','ifidf_for_words']].iterrows():
    tfidf_docs.append(get_tfidf_for_docs(row['form'],row['ifidf_for_words']))
X_train['tfidf_docs'] = tfidf_docs

In [219]:
X_train.head()

,NewsTitle,NewsFullContent,list_label,name_entity,title_tag,form,pos_tag,form_text,ifidf_for_words,tfidf_docs
332,Lợi nhuận sau thuế quý I của FPT tăng mạnh 21%,Tập đoàn FPT vừa chính thức công bố kết quả ki...,"[(tập đoàn, 1), (fpt, 1), (vừa, 0), (chính thứ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...","[Tập_đoàn, FPT, vừa, chính_thức, công_bố, kết_...","[N, Np, R, A, V, N, V, N, M, CH, M, V, P, N, A...",Tập_đoàn FPT vừa chính_thức công_bố kết_quả ki...,"{'104': 0.06216827314492683, '106': 0.05878267...",0 0.000000 1 0.000000 2 0.03595...
29,Công ty FPT bị đôn đốc nộp thuế gần 14 tỉ đồng,Nếu FPT vẫn chưa thực hiện tổng cục Hải quan s...,"[(nếu, 0), (fpt, 1), (vẫn, 0), (chưa, 1), (thự...","[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Nếu, FPT, vẫn, chưa, thực_hiện, tổng_cục, Hải...","[C, Ny, R, R, V, N, Np, R, V, L, N, V, E, N, V...",Nếu FPT vẫn chưa thực_hiện tổng_cục Hải_quan s...,"{'05': 0.08041717567771642, '10': 0.0341571841...",0 0.000000 1 0.000000 2 0.13269...
302,Thế Giới Di Động thu 20 tỷ đồng từ bán thực phẩm,Chia sẻ tại buổi gặp gỡ các nhà đầu tư chiều 5...,"[(chia sẻ, 1), (tại, 0), (buổi, 0), (gặp gỡ, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Chia_sẻ, tại, buổi, gặp_gỡ, các, nhà_đầu_tư, ...","[V, E, N, V, L, N, N, M, E, Ny, Ny, Nc, Np, Ny...",Chia_sẻ tại buổi gặp_gỡ các nhà_đầu_tư chiều 5...,"{'16': 0.04925260014415042, '19': 0.0469799266...",0 0.000000 1 0.055833 2 0.07503...
286,"Gót chân Asin của ""gã khổng lồ"" Thế giới Di động",Thành lập từ những năm 2004 khi sở hữu điện th...,"[(thành lập, 1), (từ, 0), (những, 1), (năm, 1)...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[Thành_lập, từ, những, năm, 2004, khi, sở_hữu,...","[V, E, L, N, M, N, V, N, V, Nc, V, E, R, A, N,...",Thành_lập từ những năm 2004 khi sở_hữu điện_th...,"{'000': 0.03628771343845659, '000m2': 0.025145...",0 0.000000 1 0.049263 2 0.10...
554,"Doanh thu tháng 3 của MWG vẫn tăng trưởng, đạt...",Công ty sẽ dồn lực cho Bách Hóa Xanh tạm hoã...,"[(công ty, 1), (sẽ, 1), (dồn, 1), (lực, 0), (c...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Công_ty, sẽ, dồn, lực, cho, Bách_Hoá, Xanh, t...","[N, R, V, N, E, Np, Np, A, V, V, L, N, A, E, N...",Công_ty sẽ dồn lực cho Bách_Hoá Xanh tạm hoãn ...,"{'10': 0.060573569965998074, '100': 0.02964637...",0 0.000000 1 0.036484 2 0.05902...


In [220]:
X_train = X_train.reset_index()

In [221]:
def combine_feature(X):
    features = []
    for i in range(len(X)):
        form = pd.DataFrame(X['form'][i])
        
        pos_tag = pd.DataFrame(X['pos_tag'][i])
        tfidf_docs = pd.DataFrame(X['tfidf_docs'][i])
        name_entity = pd.DataFrame(X['name_entity'][i])
        title_tag = pd.DataFrame(X['title_tag'][i])
        df_col_merged = pd.concat([form, pos_tag,tfidf_docs,name_entity,title_tag], axis=1)
        feature = df_col_merged.values.tolist()
        features.append(feature)
    X['features'] = features 

In [222]:
combine_feature(X_train)

In [223]:
X_train['features'][0]

[['Tập_đoàn', 'N', 0.0, 1, 0],
 ['FPT', 'Np', 0.0, 1, 1],
 ['vừa', 'R', 0.03595378470603885, 0, 0],
 ['chính_thức', 'A', 0.04643878430551138, 0, 0],
 ['công_bố', 'V', 0.030863181613975436, 0, 0],
 ['kết_quả', 'N', 0.1320111463780803, 0, 0],
 ['kinh_doanh', 'V', 0.0735591551843913, 0, 0],
 ['quý', 'N', 0.2943439137700736, 0, 1],
 ['I', 'M', 0.0, 0, 1],
 ['/', 'CH', 0.0, 0, 0],
 ['2017', 'M', 0.03680490446968431, 0, 0],
 ['Theo', 'V', 0.0, 0, 0],
 ['đó', 'P', 0.05653040504940959, 0, 0],
 ['doanh_thu', 'N', 0.12529927084472464, 0, 0],
 ['hợp_nhất', 'A', 0.03595378470603885, 0, 0],
 ['được', 'V', 0.06028980656614591, 0, 0],
 ['ghi_nhận', 'V', 0.10282718326362854, 0, 0],
 ['ở', 'E', 0.0, 0, 0],
 ['mức', 'N', 0.04811425870666552, 0, 0],
 ['9', 'M', 0.0, 0, 0],
 ['768', 'M', 0.07904497239552274, 0, 0],
 ['tỷ', 'M', 0.11032828534045878, 0, 0],
 ['đồng', 'Nu', 0.12387932009187075, 0, 0],
 ['tăng', 'V', 0.17935384697179552, 0, 1],
 ['14%', 'M', 0.0, 0, 0],
 ['so', 'V', 0.17448815766329548, 0, 0]

In [224]:
y_train[0]

[1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,


In [225]:
X_test.head()

,NewsTitle,NewsFullContent,list_label,name_entity,title_tag,form,pos_tag,form_text
636,"Hai nguyên phó tổng giám đốc BIDV ""chia nhau"" ...",14 giờ chiều nay 2 11 TAND TP Hà Nội đã tuyên ...,"[(14, 0), (giờ, 1), (chiều, 1), (nay, 1), (2, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[14, giờ, chiều, nay, 2, 11, TAND, TP, Hà_Nội,...","[M, Nu, N, P, M, M, Ny, Ny, Np, R, V, E, M, N,...",14 giờ chiều nay 2 11 TAND TP Hà_Nội đã tuyên_...
220,VIC - Thông qua chuyển nhượng cổ phần tại BĐS ...,Theo đó Hội đồng quản trị thống nhất chuyển ...,"[(theo, 1), (đó, 0), (hội đồng quản trị, 1), (...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[Theo, đó, Hội_đồng_quản_trị, thống_nhất, chuy...","[V, P, Np, V, V, M, M, M, N, V, V, V, N, V, M,...",Theo đó Hội_đồng_quản_trị thống_nhất chuyển_nh...
594,Masan Group bổ nhiệm Tổng giám đốc,Ông Danny Le sinh năm 1984 – người được bổ n...,"[(ông, 1), (danny le, 1), (sinh, 1), (năm, 1),...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[Ông, Danny_Le, sinh, năm, 1984, –, người, đượ...","[Nc, Np, V, N, M, CH, N, V, V, V, N, Np, V, E,...",Ông Danny_Le sinh năm 1984 – người được bổ_nhi...
429,ADB cung cấp 300 triệu USD vốn vay cho BIDV để...,Hợp đồng vay tín chấp này bao gồm khoản vay A ...,"[(hợp đồng, 1), (vay, 1), (tín chấp, 0), (này,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[Hợp_đồng, vay, tín_chấp, này, bao_gồm, khoản,...","[N, V, N, P, V, N, V, Y, V, M, M, Nu, E, Ny, V...",Hợp_đồng vay tín_chấp này bao_gồm khoản vay A ...
72,BIDV được chào bán 510 triệu cổ phiếu,Theo đó BIDV được phát hành và chào bán 510 ...,"[(theo, 1), (đó, 0), (bidv, 1), (được, 1), (ph...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, ...","[Theo, đó, BIDV, được, phát_hành, và, chào_bán...","[V, P, Ny, V, V, Cc, V, M, M, M, N, N, M, M, N...",Theo đó BIDV được phát_hành và chào_bán 510 04...


In [226]:
tfidf_matrix = tfidf.transform(X_test['form_text'])
feature_names = tfidf.get_feature_names()

def get_ifidf_for_words(text):
    tfidf_matrix= tfidf.transform([text]).todense()
    feature_index = tfidf_matrix[0,:].nonzero()[1]
    tfidf_scores = zip([feature_names[i] for i in feature_index], [tfidf_matrix[0, x] for x in feature_index])
    return dict(tfidf_scores)

X_test['ifidf_for_words'] = X_test['form_text'].apply(lambda x : get_ifidf_for_words(x))

tfidf_docs = []
for i,row in X_test[['form','ifidf_for_words']].iterrows():
    tfidf_docs.append(get_tfidf_for_docs(row['form'],row['ifidf_for_words']))
X_test['tfidf_docs'] = tfidf_docs
X_test = X_test.reset_index()
combine_feature(X_test)


In [227]:
X_test['features'][1]

[['Theo', 'V', 0.0, 0, 0],
 ['đó', 'P', 0.05632138097011974, 0, 0],
 ['Hội_đồng_quản_trị', 'Np', 0.0, 0, 0],
 ['thống_nhất', 'V', 0.16582602272492536, 0, 0],
 ['chuyển_nhượng', 'V', 0.28912644698425166, 0, 1],
 ['5', 'M', 0.0, 0, 0],
 ['25', 'M', 0.22544398693032078, 0, 0],
 ['triệu', 'M', 0.08388652666934544, 0, 0],
 ['cổ_phần', 'N', 0.0931009037212028, 0, 1],
 ['tương_ứng', 'V', 0.12589732305282839, 0, 0],
 ['trị_giá', 'V', 0.19202213331815532, 0, 0],
 ['theo', 'V', 0.11403928170651795, 0, 0],
 ['mệnh_giá', 'N', 0.18581520827541587, 0, 0],
 ['là', 'V', 0.10621462463462945, 0, 0],
 ['52', 'M', 0.16582602272492536, 0, 0],
 ['5', 'M', 0.0, 0, 0],
 ['tỷ', 'M', 0.054960170416335566, 0, 0],
 ['đồng', 'Nu', 0.05289482997271594, 0, 0],
 ['chiếm', 'V', 0.11089143684046283, 0, 0],
 ['26', 'M', 0.12101069041914658, 0, 0],
 ['25%', 'M', 0.0, 0, 0],
 ['vốn_điều_lệ', 'N', 0.11563400738536556, 0, 0],
 ['của', 'E', 0.046900387274575225, 0, 0],
 ['CTCP', 'Ny', 0.0, 1, 0],
 ['Đầu_tư', 'V', 0.0, 1, 0],

In [228]:
len(y_test[1])

48

In [229]:
X_train['y'] = y_train
X_test['y'] = y_test

X_train[['features','y']].to_csv('training.csv')
X_test[['features','y']].to_csv('testing.csv')

In [230]:
X_train['y']

0      [1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, ...
1      [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, ...
2      [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, ...
3      [1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, ...
4      [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, ...
                             ...                        
514    [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, ...
515    [1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, ...
516    [1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, ...
517    [0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, ...
518    [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, ...
Name: y, Length: 519, dtype: object

In [231]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    tfidf = doc[i][2]
    ner = doc[i][3]
    title = doc[i][4]
    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isner=%s' % ner,
        'tfidf=%f' % tfidf,
        'word.istitle=%s' % title,
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        tfidf1 = doc[i-1][2]
        ner1 = doc[i-1][3]
        title1 = doc[i-1][4]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.isner=%s' % ner1,
            '-1:tfidf=%f'  % tfidf1,
            '-1:word.istitle=%s' % title1,
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        tfidf1 = doc[i+1][2]
        ner1 = doc[i+1][3]
        title1 = doc[i+1][4]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.isner=%s' % ner1,
            '+1:tfidf=%f'  % tfidf1,
            '+1:word.istitle=%s' % title1,
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [232]:
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]


X_train1 = [extract_features(doc) for doc in X_train['features']]
X_test1 = [extract_features(doc) for doc in X_test['features']]

In [233]:
X_train1[0]

word.lower=với',
  '+1:word.isner=0',
  '+1:tfidf=0.190884',
  '+1:word.istitle=0',
  '+1:postag=E'],
 ['bias',
  'word.lower=với',
  'word.isner=0',
  'tfidf=0.190884',
  'word.istitle=0',
  'postag=E',
  '-1:word.lower=quý',
  '-1:word.isner=0',
  '-1:tfidf=0.294344',
  '-1:word.istitle=1',
  '-1:postag=N',
  '+1:word.lower=mức',
  '+1:word.isner=0',
  '+1:tfidf=0.048114',
  '+1:word.istitle=0',
  '+1:postag=N'],
 ['bias',
  'word.lower=mức',
  'word.isner=0',
  'tfidf=0.048114',
  'word.istitle=0',
  'postag=N',
  '-1:word.lower=với',
  '-1:word.isner=0',
  '-1:tfidf=0.190884',
  '-1:word.istitle=0',
  '-1:postag=E',
  '+1:word.lower=tăng',
  '+1:word.isner=0',
  '+1:tfidf=0.179354',
  '+1:word.istitle=1',
  '+1:postag=V'],
 ['bias',
  'word.lower=tăng',
  'word.isner=0',
  'tfidf=0.179354',
  'word.istitle=1',
  'postag=V',
  '-1:word.lower=mức',
  '-1:word.isner=0',
  '-1:tfidf=0.048114',
  '-1:word.istitle=0',
  '-1:postag=N',
  '+1:word.lower=33%',
  '+1:word.isner=0',
  '+1:tfi

In [234]:
def convert(y):
    y_ = y.copy()
    for i in range(len(y)):
        if y_[i] == 0:
            y_[i] = 'N'
        else:
            y_[i] = 'I'
    return y_



In [235]:
for i in range(len(y_train)):
    y_train[i] = convert(y_train[i])
for i in range(len(y_test)):
    y_test[i] = convert(y_test[i])

In [239]:
from sklearn_crfsuite import CRF

In [240]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True
)
crf.fit(X_train1, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=200)

In [242]:
y_pred = crf.predict(X_test1)

In [248]:
import numpy as np
from sklearn.metrics import classification_report


crf.score(X_test1, y_test)

0.9908300341758874